In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')

In [2]:
#  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 75 76 77 78 79 80 81 82 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 164 165 167 168 169 170 171 172 173 174 175 176 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 277 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 368 369 370 371 372 374 375 376 377 378 379 380 381 382 383 385 386 387 388 389 390 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 907 908 909 910 911 912 913 914 915 916 917 918 919 920 922 923 925 926 927 928 929 930 931 932 933 934 935 936 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 957 958 959 960 961 963 964 965 966 967 968 969 971 972 973 974 975 976 977 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1035

In [3]:
# D-optimal을 통해 최적화 된 행 전처리
rows_100="34 52 68 74 77 85 115 117 120 123 129 146 160 161 167 178 191 193 196 198 201 212 221 234 238 252 257 258 261 272 288 295 306 316 318 334 341 358 377 402 409 418 421 426 432 439 451 464 476 483 491 498 506 509 517 532 539 540 548 549 579 586 614 626 664 670 679 683 686 694 705 732 734 738 742 744 770 772 781 803 804 808 825 841 853 872 878 901 918 947 949 958 963 973 981 1001 1011 1014 1028 1032"   
rows_100 = re.sub(' +', ',', rows_100)
rows_100 = rows_100.split(",")
rows_100 = [int(i)-1 for i in rows_100]
len(rows_100)

100

In [4]:
# D-optimal을 통해 최적화 된 행 전처리
rows_200="22 29 37 38 62 66 80 93 100 103 106 110 112 122 125 128 134 137 148 153 162 169 170 178 179 186 192 195 206 207 208 210 212 218 221 231 234 236 238 241 244 245 247 248 252 254 256 257 258 259 261 272 273 275 280 282 289 308 309 312 317 319 326 333 339 340 342 343 351 370 372 373 377 378 382 385 388 391 395 402 416 418 427 428 429 430 435 446 452 454 464 465 474 476 478 483 488 490 491 492 495 499 501 503 506 515 517 528 529 536 540 542 545 553 557 561 583 585 590 612 616 619 620 621 631 636 645 650 659 660 670 671 672 679 681 683 692 694 700 703 705 712 714 717 723 725 735 738 745 751 755 757 759 777 789 794 795 796 797 804 808 810 827 829 833 839 845 846 850 853 855 864 874 889 896 898 905 906 912 920 923 935 941 952 954 968 972 976 978 979 985 994 995 1004 1011 1015 1016 1018 1022 1035"   
rows_200 = re.sub(' +', ',', rows_200)
rows_200 = rows_200.split(",")
rows_200 = [int(i)-1 for i in rows_200]
len(rows_200)

200

In [5]:
# D-optimal을 통해 최적화 된 행 전처리
rows_300="8 9 12 17 25 26 34 41 43 54 55 56 57 58 66 67 68 74 76 77 83 90 94 100 103 107 111 112 117 118 119 131 133 142 145 146 151 162 167 168 170 172 173 176 177 178 179 183 189 190 192 202 203 204 207 210 212 214 223 225 226 230 233 235 238 240 243 244 247 256 259 261 266 267 268 272 276 279 280 281 287 288 289 290 291 292 293 295 300 304 306 311 312 314 315 332 337 341 344 354 355 356 358 362 364 371 373 374 376 378 384 385 391 393 397 399 400 402 405 410 413 416 423 424 427 428 429 430 431 436 445 446 448 451 463 464 465 467 471 474 478 481 482 483 484 490 492 494 496 509 517 523 525 526 528 539 540 542 553 558 561 563 572 578 579 585 587 589 591 594 596 598 599 609 610 611 613 616 617 618 630 635 637 644 646 649 651 654 656 657 658 659 661 664 669 676 679 684 685 692 693 695 699 701 720 724 726 727 728 729 730 731 733 739 744 747 748 752 755 759 765 767 770 773 776 778 782 783 786 787 789 792 797 803 804 805 806 807 808 820 824 825 826 829 832 836 840 844 845 850 858 860 864 870 877 885 898 901 902 905 910 917 918 936 938 939 943 945 949 950 958 960 962 964 967 968 975 980 981 985 989 992 993 997 999 1002 1005 1006 1007 1011 1012 1014 1017 1020 1021 1026 1027 1028 1030 1035"   
rows_300 = re.sub(' +', ',', rows_300)
rows_300 = rows_300.split(",")
rows_300 = [int(i)-1 for i in rows_300]
len(rows_300)

300

In [6]:
# D-optimal을 통해 최적화 된 행 전처리
rows_400="6 9 13 14 15 18 25 28 29 30 32 33 37 39 42 43 46 52 57 58 59 60 68 70 76 77 78 79 80 84 85 86 92 93 96 97 101 103 104 106 107 109 113 114 116 117 120 121 130 132 133 136 137 141 142 145 148 150 153 154 159 163 164 165 171 175 176 178 180 183 186 188 191 192 193 195 197 201 203 205 208 210 212 217 220 222 224 227 228 230 231 232 234 237 243 245 246 250 251 255 257 264 266 269 273 276 280 284 287 294 296 301 303 304 305 307 311 314 315 317 318 319 320 330 333 340 344 348 351 353 354 355 356 358 361 362 363 364 366 367 373 374 375 381 384 385 387 388 389 394 398 400 402 404 405 410 413 415 416 421 422 425 426 431 432 434 437 447 448 450 451 452 453 454 456 458 460 461 462 464 466 468 474 477 478 480 484 488 492 494 499 504 507 508 509 511 514 518 523 524 527 528 530 531 533 536 539 541 542 545 547 550 551 552 553 555 556 558 559 560 564 567 569 571 572 580 582 583 586 589 590 591 593 595 598 603 604 606 607 608 614 615 616 617 619 620 625 626 634 635 636 639 641 643 646 649 653 654 655 661 662 665 666 670 672 675 676 680 685 687 688 690 692 693 694 695 697 698 700 703 705 707 708 709 710 715 719 720 725 730 731 734 736 738 739 742 744 745 747 748 749 751 756 758 759 760 761 764 766 772 776 780 781 787 789 790 792 797 800 803 805 806 807 809 810 812 813 815 823 824 827 829 838 840 844 848 853 854 857 858 859 860 865 868 876 877 881 882 884 885 887 891 896 900 901 902 905 907 910 911 914 915 926 929 934 936 937 941 942 945 948 949 955 957 960 961 965 967 971 973 976 977 978 980 981 983 986 988 992 995 1003 1005 1008 1009 1010 1018 1022 1023 1029 1031"   
rows_400 = re.sub(' +', ',', rows_400)
rows_400 = rows_400.split(",")
rows_400 = [int(i)-1 for i in rows_400]
len(rows_400)

400

In [7]:
# D-optimal을 통해 최적화 된 행 전처리
rows_500="1 2 3 6 8 10 15 16 17 20 25 30 33 34 35 38 41 42 43 46 47 48 49 53 54 55 56 60 64 65 66 67 68 69 70 71 72 73 75 76 78 79 82 83 84 87 88 92 99 105 106 108 119 126 127 129 131 132 137 139 142 143 144 148 149 151 157 158 160 161 163 166 167 175 178 179 180 182 183 188 189 191 192 193 194 198 200 201 202 204 212 213 215 218 219 228 229 230 233 238 241 243 245 246 247 250 252 254 261 263 265 267 268 269 270 278 280 281 283 284 285 286 287 290 291 294 295 297 298 300 301 303 308 310 311 313 315 316 318 319 321 323 328 330 331 335 337 340 344 345 346 349 350 353 354 355 358 360 361 362 364 365 367 368 369 373 375 379 381 383 384 386 390 391 392 394 396 397 398 399 401 402 403 405 407 410 412 413 416 418 419 420 422 423 424 426 428 432 433 434 441 443 445 447 450 451 452 454 459 461 462 463 464 465 466 467 471 472 473 474 476 478 479 480 481 482 483 484 486 488 489 491 493 494 495 496 498 502 507 509 511 513 517 520 523 525 528 529 530 531 533 534 537 538 545 546 547 548 554 557 559 562 566 568 572 574 576 578 580 581 582 586 590 591 592 593 595 596 600 604 605 607 609 611 612 613 615 616 619 620 621 622 627 629 632 633 634 636 638 639 642 644 647 650 652 653 654 655 656 657 658 660 664 665 666 667 668 674 675 679 683 684 685 687 688 690 693 698 699 703 705 710 711 712 713 714 715 717 718 719 720 723 725 728 731 732 733 734 735 736 737 738 740 744 746 747 749 752 753 755 758 760 761 762 763 764 765 767 768 772 774 776 780 781 782 783 785 787 788 791 793 795 797 798 800 801 802 804 805 812 814 815 816 821 822 824 826 828 829 833 839 841 843 845 847 849 850 851 854 856 860 863 867 869 870 874 875 876 877 878 879 881 882 887 888 892 893 894 898 901 902 903 904 906 908 909 910 911 915 918 919 920 921 923 925 926 928 929 935 936 937 947 948 949 951 952 953 954 956 957 958 960 962 963 964 965 967 970 972 977 980 981 983 987 988 990 991 994 995 1000 1003 1004 1005 1006 1011 1012 1013 1014 1015 1016 1018 1019 1020 1022 1023 1024 1028 1033 1034 1035"   
rows_500 = re.sub(' +', ',', rows_500)
rows_500 = rows_500.split(",")
rows_500 = [int(i)-1 for i in rows_500]
len(rows_500)

500

In [8]:
# D-optimal을 통해 최적화 된 행 전처리
rows_600="1 2 4 9 11 14 17 18 22 23 24 25 26 31 35 36 37 38 39 40 43 44 45 49 50 51 52 53 56 57 58 59 61 62 63 64 66 67 68 69 70 71 72 73 74 75 76 77 79 80 82 85 89 91 92 94 96 97 100 101 102 103 104 105 106 108 109 114 115 116 117 118 120 122 124 126 127 128 129 132 133 135 139 140 141 144 145 147 148 149 151 152 154 155 156 157 159 160 162 163 164 166 167 168 169 170 171 173 176 178 179 181 182 183 184 185 186 189 192 193 194 195 197 202 208 210 216 217 220 221 223 224 225 226 227 229 231 233 234 235 236 237 238 239 240 242 243 245 247 248 249 252 253 254 255 256 257 259 262 264 266 269 270 271 274 282 283 285 286 288 289 290 294 295 296 297 298 300 303 307 308 311 317 318 322 325 329 331 333 334 339 340 342 345 348 349 350 351 353 354 355 356 360 362 364 365 366 367 368 370 371 373 375 376 377 378 382 384 388 389 391 394 397 398 400 401 402 403 404 405 407 408 409 410 411 412 413 415 420 422 424 428 430 432 433 435 437 438 439 441 442 443 445 446 447 448 450 453 454 455 456 459 463 464 466 469 470 471 472 474 478 479 480 481 482 484 485 487 489 490 494 496 497 499 500 501 504 507 508 511 514 517 518 519 520 521 522 525 527 528 529 532 533 534 537 539 541 543 545 546 550 551 552 553 555 557 558 560 561 562 564 565 567 568 569 570 574 575 578 579 580 581 582 586 588 589 590 592 593 594 596 597 598 599 600 601 603 605 607 608 610 613 615 616 619 620 621 623 624 625 626 628 629 630 631 632 633 636 637 638 639 640 641 642 643 644 645 647 651 652 654 656 659 661 663 665 670 671 672 676 677 679 680 681 682 683 686 687 691 695 696 697 698 699 700 701 702 703 704 709 710 711 712 714 715 717 718 721 723 724 727 728 729 736 737 739 740 742 744 745 746 747 748 749 751 752 754 756 759 760 761 762 763 767 768 769 771 772 773 774 776 777 781 782 785 786 787 789 790 791 794 795 796 797 798 801 802 804 805 806 810 811 812 814 815 818 819 820 822 823 825 826 827 829 832 833 837 838 841 845 846 847 848 852 853 854 856 858 859 861 862 864 865 868 871 874 875 878 880 882 885 886 887 888 889 891 892 893 897 898 903 904 905 906 909 911 913 915 916 917 918 921 922 924 926 931 932 936 937 939 940 941 942 943 944 948 950 953 954 955 956 959 960 963 964 965 966 969 970 971 972 973 974 976 978 979 981 983 984 986 992 993 994 995 996 997 998 999 1000 1002 1003 1004 1006 1007 1009 1010 1011 1012 1014 1016 1017 1020 1021 1022 1023 1024 1025 1033 1034 1035"   
rows_600 = re.sub(' +', ',', rows_600)
rows_600 = rows_600.split(",")
rows_600 = [int(i)-1 for i in rows_600]
len(rows_600)

600

In [9]:
# D-optimal을 통해 최적화 된 행 전처리
rows_700="1 2 4 6 7 8 9 10 11 12 13 14 15 17 20 21 22 26 27 28 29 30 33 34 35 36 37 38 41 46 47 48 49 50 51 56 60 61 62 63 68 69 71 73 75 79 81 84 85 87 88 90 91 92 93 96 98 99 100 101 104 106 108 109 111 112 114 116 117 118 120 122 123 124 128 129 130 131 132 133 134 136 138 139 142 143 144 145 146 147 148 150 151 152 153 154 155 156 158 160 161 165 166 167 169 171 172 173 174 176 177 178 179 185 186 188 189 191 192 194 195 196 198 200 201 203 206 209 210 211 212 213 214 216 220 221 226 228 229 230 231 232 235 236 237 238 239 240 242 243 245 246 247 249 252 254 256 257 259 261 262 263 264 265 266 267 270 271 272 275 277 278 279 280 281 282 284 286 288 289 290 291 294 297 299 304 305 306 307 311 312 313 314 315 317 318 320 321 326 327 328 329 331 333 334 335 339 342 345 346 347 348 349 350 351 352 354 356 357 359 362 363 364 366 367 368 369 370 372 375 377 379 380 381 382 383 384 385 386 387 388 390 391 392 393 394 395 396 398 399 400 402 403 404 405 406 407 408 409 410 411 412 414 415 417 420 422 423 425 426 428 430 431 432 433 435 437 438 440 441 443 444 445 446 449 450 453 454 455 456 460 462 463 464 465 466 467 468 469 470 471 472 474 475 476 477 478 479 480 481 482 483 484 485 486 488 489 492 494 496 498 499 500 501 502 503 504 507 508 510 511 513 514 515 516 518 519 520 521 522 523 524 525 526 527 528 530 533 534 535 537 538 543 544 548 550 551 552 553 554 555 556 557 558 560 563 564 566 569 575 576 577 578 580 583 584 585 586 587 590 592 593 594 595 596 598 599 601 602 605 606 608 609 610 611 613 617 618 619 620 623 624 625 629 630 631 633 634 635 636 637 638 639 640 642 643 644 647 648 649 650 651 652 653 655 656 657 659 660 661 662 663 666 668 669 670 671 672 675 676 678 679 681 682 683 686 687 688 689 690 691 692 693 694 695 697 699 700 701 703 705 706 707 708 709 710 711 712 714 715 717 718 719 720 721 722 723 724 726 727 728 730 731 732 733 734 736 737 738 739 740 741 742 743 744 745 746 747 749 750 751 752 753 754 755 756 758 759 760 761 762 763 764 767 769 770 772 773 774 776 777 780 781 782 783 784 785 787 788 790 791 792 794 797 800 801 803 804 805 806 809 810 811 812 813 814 816 817 818 819 820 821 823 824 825 827 828 829 830 831 832 836 837 838 840 841 842 843 845 846 848 849 850 851 852 854 856 860 861 862 863 864 866 867 868 870 871 872 873 874 875 876 878 880 881 882 886 887 888 890 892 895 896 897 899 901 903 905 906 908 909 910 911 912 914 915 916 918 919 920 922 925 927 928 930 931 932 933 935 936 937 939 940 942 943 944 945 946 947 948 949 950 951 952 954 955 956 957 960 961 963 964 966 967 968 969 970 972 973 974 975 979 981 983 984 985 987 988 989 990 993 996 997 1001 1003 1004 1005 1006 1007 1008 1009 1011 1012 1013 1014 1017 1018 1019 1021 1023 1025 1026 1027 1028 1030 1031 1032 1033 1034 1035"   
rows_700 = re.sub(' +', ',', rows_700)
rows_700 = rows_700.split(",")
rows_700 = [int(i)-1 for i in rows_700]
len(rows_700)

700

In [10]:
# D-optimal을 통해 최적화 된 행 전처리
rows_800="2 3 5 7 8 9 10 11 13 15 18 19 21 23 24 27 28 29 30 31 32 34 35 37 39 41 43 44 45 46 47 48 52 53 54 55 59 60 61 62 63 64 67 68 69 70 71 72 73 74 76 77 80 82 83 84 85 86 87 88 89 92 93 94 95 96 97 98 99 100 101 102 104 107 108 109 110 112 113 114 115 116 117 118 119 121 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 140 141 142 144 145 146 147 148 149 153 154 155 159 160 161 162 163 164 165 166 168 170 171 173 175 176 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 194 196 197 198 199 200 201 202 203 204 206 208 209 210 211 212 213 216 217 218 219 220 221 222 223 224 225 226 227 228 230 231 232 233 234 235 236 237 238 239 240 241 242 243 246 247 248 249 250 251 252 253 255 256 257 259 260 261 262 263 265 266 267 268 269 270 271 272 273 274 275 276 277 278 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 296 297 299 300 301 303 304 306 307 308 309 310 311 312 313 314 316 318 319 320 321 322 323 324 327 328 330 332 333 334 335 336 337 338 340 341 342 343 344 345 346 347 348 349 350 351 352 353 355 356 357 358 359 360 361 362 363 366 367 368 369 370 371 372 373 374 375 377 378 379 380 381 382 383 384 385 386 388 389 390 392 393 394 395 396 397 399 400 403 406 408 409 411 412 413 414 415 417 418 419 423 424 426 427 428 429 432 434 435 436 437 440 441 444 445 446 447 448 449 450 451 454 456 457 458 459 460 461 462 463 464 465 467 468 469 471 473 475 477 479 480 481 482 484 485 486 487 488 489 490 491 493 494 495 496 497 498 500 501 502 503 504 505 506 507 509 510 511 514 515 516 517 518 519 522 523 524 525 527 528 529 531 532 533 534 535 536 539 540 541 542 543 545 546 547 548 550 553 554 556 557 558 561 563 564 565 566 567 568 569 570 572 573 574 575 576 577 578 579 580 581 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 602 603 604 606 607 609 610 611 612 613 614 615 616 618 620 622 624 625 626 627 628 629 630 632 633 634 635 637 638 639 640 641 643 644 645 646 648 649 650 651 653 654 655 656 657 659 660 663 664 665 666 667 669 670 671 674 675 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 696 697 698 699 700 701 702 703 705 706 707 709 710 711 712 713 714 715 718 719 721 723 724 726 729 730 731 732 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 755 756 757 759 761 762 763 764 765 766 767 768 769 770 771 773 774 776 777 779 780 781 782 784 786 787 789 790 794 795 797 798 799 800 801 802 804 806 807 808 809 810 813 815 816 817 819 821 822 824 825 826 827 829 830 832 833 836 837 838 840 841 842 843 845 846 847 848 849 850 852 853 854 855 856 858 860 861 862 864 866 867 868 869 870 871 872 873 874 875 876 878 879 880 881 883 884 885 887 888 889 890 893 897 898 901 902 903 905 906 907 908 910 911 912 913 914 916 917 918 919 921 922 923 924 925 926 928 931 933 934 935 936 937 938 939 940 941 944 945 947 948 949 950 951 955 956 957 958 959 961 962 964 965 966 967 970 971 972 973 974 978 980 981 982 984 985 986 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1008 1009 1011 1013 1015 1016 1018 1019 1020 1021 1022 1023 1027 1028 1029 1030 1031 1032 1033 1034 1035"   
rows_800 = re.sub(' +', ',', rows_800)
rows_800 = rows_800.split(",")
rows_800 = [int(i)-1 for i in rows_800]
len(rows_800)

800

In [11]:
# D-optimal을 통해 최적화 된 행 전처리
rows_900="1 2 4 5 6 7 8 10 12 13 14 15 16 17 19 21 22 23 24 25 26 27 28 30 31 33 34 35 37 38 40 41 42 43 45 46 47 48 49 50 51 52 53 54 55 56 57 59 60 61 62 63 64 65 66 67 68 70 71 72 73 76 77 79 80 81 82 83 84 85 86 87 88 90 92 93 95 96 97 98 99 100 101 102 105 106 107 109 110 112 113 114 115 116 117 119 120 122 123 124 125 126 127 128 129 130 131 133 134 135 136 137 138 139 140 141 142 143 147 148 149 150 152 153 155 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 175 176 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 211 212 213 214 215 216 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 256 257 258 259 260 261 262 263 264 265 266 267 268 270 271 272 273 274 275 276 277 278 279 280 281 283 284 285 286 287 288 289 290 291 292 293 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 322 323 324 325 326 327 328 329 330 331 333 334 335 336 337 338 339 340 341 342 343 344 345 346 349 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 368 369 371 372 373 374 375 376 377 378 379 380 381 382 383 385 386 387 388 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 415 416 418 419 420 421 422 424 425 426 427 428 429 431 432 433 435 436 437 438 439 440 441 442 443 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 464 465 466 467 468 470 471 473 474 475 476 477 480 481 482 484 485 486 487 488 489 490 491 493 494 495 496 497 498 499 500 502 503 504 505 506 507 508 509 510 511 513 514 516 517 518 520 521 522 523 524 525 526 527 528 530 531 532 533 534 535 536 537 538 539 540 541 543 544 545 546 547 548 549 550 551 552 553 554 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 574 575 577 578 579 580 581 582 583 584 585 586 588 589 590 591 592 593 594 595 597 598 599 600 602 603 605 606 607 608 609 610 611 612 613 616 617 618 619 620 622 623 625 626 627 628 629 630 634 635 636 637 638 639 640 641 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 659 660 661 662 663 664 665 667 668 669 670 671 672 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 698 699 700 701 703 704 705 706 707 709 710 712 714 715 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 738 740 741 742 743 744 745 746 747 748 750 751 752 754 755 756 757 758 759 760 761 762 763 764 765 766 767 769 770 771 772 773 774 775 776 778 779 780 782 784 785 786 787 788 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 840 841 843 844 845 846 847 848 849 850 851 852 853 854 855 857 858 859 861 862 863 864 865 866 868 869 870 871 872 873 876 878 879 881 882 883 884 886 888 889 890 891 892 893 894 896 898 899 900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 925 926 927 928 929 931 932 933 934 935 936 938 939 940 941 942 943 944 946 947 948 949 950 951 952 954 955 956 957 958 959 960 961 962 963 964 965 966 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 990 991 992 994 995 996 997 1000 1002 1003 1004 1005 1006 1007 1008 1009 1010 1012 1013 1014 1015 1017 1019 1020 1021 1022 1023 1024 1025 1026 1027 1029 1030 1031 1032 1033 1034 1035"   
rows_900 = re.sub(' +', ',', rows_900)
rows_900 = rows_900.split(",")
rows_900 = [int(i)-1 for i in rows_900]
len(rows_900)

900

In [12]:
# D-optimal을 통해 최적화 된 행 전처리
rows_1000="1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 75 76 77 78 79 80 81 82 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 164 165 167 168 169 170 171 172 173 174 175 176 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 277 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 368 369 370 371 372 374 375 376 377 378 379 380 381 382 383 385 386 387 388 389 390 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 907 908 909 910 911 912 913 914 915 916 917 918 919 920 922 923 925 926 927 928 929 930 931 932 933 934 935 936 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 957 958 959 960 961 963 964 965 966 967 968 969 971 972 973 974 975 976 977 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1035"   
rows_1000 = re.sub(' +', ',', rows_1000)
rows_1000 = rows_1000.split(",")
rows_1000 = [int(i)-1 for i in rows_1000]
len(rows_1000)

1000

### 테스트셋 생성

In [13]:
# # 원본 데이터에서 D-optimal로 최적화 되지 않은 dataframe만 추출
# testset = df.drop(index=rows, axis=0)
# testset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv",index=False)

### 훈련셋 생성

In [14]:
trainset = df.iloc[rows_100,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-100-using-doe.csv",index=False)

In [15]:
trainset = df.iloc[rows_200,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-200-using-doe.csv",index=False)

In [16]:
trainset = df.iloc[rows_300,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-300-using-doe.csv",index=False)

In [17]:
trainset = df.iloc[rows_400,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-400-using-doe.csv",index=False)

In [18]:
trainset = df.iloc[rows_500,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-500-using-doe.csv",index=False)

In [19]:
trainset = df.iloc[rows_600,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-600-using-doe.csv",index=False)

In [20]:
trainset = df.iloc[rows_700,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-700-using-doe.csv",index=False)

In [21]:
trainset = df.iloc[rows_800,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-800-using-doe.csv",index=False)

In [22]:
trainset = df.iloc[rows_900,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-900-using-doe.csv",index=False)

In [23]:
trainset = df.iloc[rows_1000,:]
trainset.to_csv("/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset-for-compare-random/train-data-1000-using-doe.csv",index=False)